In [1]:
import numpy as np
import keras
import pandas as pd
import joblib
from keras import Sequential
from first_test import EVENT_DICT
import first_test
import tensorflow as tf
from sklearn.metrics import classification_report

2025-04-28 21:44:55.477089: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-28 21:44:55.477480: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 21:44:55.479708: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 21:44:55.486441: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745898295.498618   42522 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745898295.50

In [2]:
dev_x = np.load('dwt_najafi/dev_naj_x.npy', allow_pickle=True)
dev_y = np.load('dwt_najafi/dev_naj_y.npy', allow_pickle=True)
eval_x = np.load('dwt_najafi/eval_naj_x.npy', allow_pickle=True)
eval_y = np.load('dwt_najafi/eval_naj_y.npy', allow_pickle=True)
train_x = np.load('dwt_najafi/train_naj_x.npy', allow_pickle=True)
train_y = np.load('dwt_najafi/train_naj_y.npy', allow_pickle=True)

In [3]:
print(dev_y)
print(train_x.shape)

[0 0 0 ... 1 1 1]
(13192, 22, 40)


In [4]:
dev_x_tensor = tf.convert_to_tensor(dev_x)
train_x_tensor = tf.convert_to_tensor(train_x)
eval_x_tensor = tf.convert_to_tensor(eval_x)

E0000 00:00:1745898309.988092   42522 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1745898309.991750   42522 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
"""dev_y.apply(lambda x: EVENT_DICT[x])
eval_y.apply(lambda x: EVENT_DICT[x])
train_y.apply(lambda x: EVENT_DICT[x])"""

In [5]:
def create_model():
    model = keras.models.Sequential([
        keras.Input(shape=(22,40)),
        keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.MaxPooling1D(pool_size=2),
        keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Conv1D(filters=64, kernel_size=1, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Bidirectional(keras.layers.LSTM(200, return_sequences=True, dropout=0.2093)),
        keras.layers.Bidirectional(keras.layers.LSTM(200, return_sequences=True, dropout=0.2093)),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.GlobalAveragePooling1D(),
        keras.layers.Dense(12, activation='softmax')
    ])
    return model
model_cnn = create_model()
model_cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 22, 64)         │         7,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 11, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 11, 64)         │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 11, 64)         │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 11, 400)        │       424,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 11, 400)        │       961,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 11, 512)        │       205,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 12)             │         6,156 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,617,228 (6.17 MB)

 Trainable params: 1,617,228 (6.17 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model_cnn.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, ema_momentum=0.95), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
model_cnn.fit(train_x_tensor, train_y, epochs=20, validation_data=(eval_x_tensor, eval_y), verbose=2)

Epoch 1/20
413/413 - 16s - 40ms/step - accuracy: 0.5422 - loss: 1.6915 - val_accuracy: 9.1855e-04 - val_loss: 2.6512
Epoch 2/20
413/413 - 12s - 29ms/step - accuracy: 0.5466 - loss: 1.5676 - val_accuracy: 9.1855e-04 - val_loss: 2.4698
Epoch 3/20
413/413 - 12s - 29ms/step - accuracy: 0.5465 - loss: 1.5858 - val_accuracy: 9.1855e-04 - val_loss: 2.6375
Epoch 4/20
413/413 - 12s - 29ms/step - accuracy: 0.5466 - loss: 1.5935 - val_accuracy: 9.1855e-04 - val_loss: 2.5656
Epoch 5/20
413/413 - 12s - 29ms/step - accuracy: 0.5466 - loss: 1.5615 - val_accuracy: 9.1855e-04 - val_loss: 2.5639
Epoch 6/20
413/413 - 12s - 29ms/step - accuracy: 0.5466 - loss: 1.5570 - val_accuracy: 9.1855e-04 - val_loss: 2.6968
Epoch 7/20
413/413 - 12s - 29ms/step - accuracy: 0.5465 - loss: 1.5865 - val_accuracy: 9.1855e-04 - val_loss: 2.4930
Epoch 8/20
413/413 - 12s - 29ms/step - accuracy: 0.5466 - loss: 1.5496 - val_accuracy: 9.1855e-04 - val_loss: 2.5743
Epoch 9/20
413/413 - 12s - 29ms/step - accuracy: 0.5465 - loss: 

In [8]:
def create_naj_model():
    model = Sequential([
        keras.Input(shape=(22, 40)),
        keras.layers.Bidirectional(keras.layers.LSTM(22, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(22, return_sequences=True)),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.Dense(200, activation='relu'),
        keras.layers.GlobalAveragePooling1D(),
        keras.layers.Dense(12, activation='softmax'),
    ])
    return model
model = create_naj_model()
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional) │ (None, 22, 44)         │        11,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 22, 44)         │        11,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 22, 512)        │        23,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 22, 200)        │       102,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 200)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 12)             │         2,412 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 150,932 (589.58 KB)

 Trainable params: 150,932 (589.58 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(train_x_tensor, train_y, epochs=30, batch_size=150, validation_data=(eval_x_tensor, eval_y), verbose=2)

Epoch 1/30
88/88 - 6s - 67ms/step - accuracy: 0.6438 - loss: 1.1298 - val_accuracy: 0.0381 - val_loss: 2.5581
Epoch 2/30
88/88 - 2s - 28ms/step - accuracy: 0.6931 - loss: 0.8854 - val_accuracy: 9.1855e-04 - val_loss: 2.9843
Epoch 3/30
88/88 - 3s - 28ms/step - accuracy: 0.7245 - loss: 0.7984 - val_accuracy: 0.0040 - val_loss: 3.1702
Epoch 4/30
88/88 - 3s - 29ms/step - accuracy: 0.7458 - loss: 0.7259 - val_accuracy: 0.1067 - val_loss: 3.0594
Epoch 5/30
88/88 - 3s - 29ms/step - accuracy: 0.7802 - loss: 0.6312 - val_accuracy: 0.0181 - val_loss: 3.3123
Epoch 6/30
88/88 - 3s - 28ms/step - accuracy: 0.7859 - loss: 0.5988 - val_accuracy: 0.0378 - val_loss: 3.2845
Epoch 7/30
88/88 - 3s - 30ms/step - accuracy: 0.7997 - loss: 0.5542 - val_accuracy: 0.0326 - val_loss: 3.4752
Epoch 8/30
88/88 - 3s - 29ms/step - accuracy: 0.7994 - loss: 0.5550 - val_accuracy: 0.0730 - val_loss: 3.6898
Epoch 9/30
88/88 - 2s - 28ms/step - accuracy: 0.8211 - loss: 0.4982 - val_accuracy: 0.0383 - val_loss: 3.4675
Epoch 

In [11]:
cnn_pred = model_cnn.predict(dev_x_tensor)
pred = model.predict(dev_x_tensor)

260/260 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
260/260 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


In [13]:
report_cnn = model_cnn.evaluate(dev_x_tensor, dev_y)

260/260 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0436 - loss: 2.3824


In [14]:
report = model.evaluate(dev_x_tensor, dev_y)

260/260 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1877 - loss: 7.2303
